# VAMPIRE WORKFLOW

## Purpose: To split tile scans, pick training and testing image sets, and in the future run the full VAMPIRE workflow

*Step 1: Import necessary packages*

In [ ]:
import shutil, os

from glob import glob

import numpy as np
import pandas as pd
from skimage import io
import matplotlib.pyplot as plt
from PIL import Image
from numpy.linalg import inv
import image_slicer
from sklearn.model_selection import train_test_split


%matplotlib inline

*Step 2: User Inputs*

Manual Step:
1. Move/Download the images for testing and training into a new folder
2. Rename images to insure they include the condition somewhere in them
3. Add a folder for each of your stains into the folder created in step 1
4. Input the name of that folder into 'folder_location' below
5. Input the names of the nuclear stain into 'stain1' and the cell stain into 'stain2' below
6. Insert your testing conditions into 'condition1' and 'condition 2' below
7. Insert the number of slices that you want to split each image into in 'slice number'
8. Add a folder labeled 'train' to your desktop
9. Add a folder labeled 'test' to your desktop
10. Within folder 'test' create a folder for each of your conditions

In [ ]:
#file names should be in the current working directory
folder_location = '/Users/hhelmbre/Desktop/Kate_images'

stain1 = 'dapi'
stain2 = 'iba'

#conditions are our four regions
conditions = np.arange([1,15,1])
conditions

file_type_init = '.tif'
file_type_new = '.png'

slice_number = 4
random_state_num = 12

In [134]:
def folder_cleaner(folder, image_type):
    k=0
    for files in folder:
        if image_type in str(files):
            k+=1
        else:
            folder = np.delete(folder, np.argwhere(folder == str(files)))
    return folder

*Step 5: Split the Image(s) into tiles*

In [135]:
arr = os.listdir(folder_location)
file_list = np.asarray(arr)
file_list = folder_cleaner(file_list, file_type_init)

In [136]:
file_list

array(['31.13_CA3_quant.tif', '31.10_CA3_quant.tif', '31.9_CA1_quant.tif',
       '31.13_CA2_quant.tif', '31.10_CA2_quant.tif', '31.13_DG_quant.tif',
       '31.13_CA1_quant.tif', '31.9_DG_quant.tif', '31.9_CA2_quant.tif',
       '31.10_DG_quant.tif', '31.10_CA1_quant.tif', '31.9_CA3_quant.tif'],
      dtype='<U19')

In [137]:
for files in file_list:
    im=io.imread(str(folder_location + '/' + files))
    channel1 = im[0, :, :]
    channel2= im[1, :, :]
    filename = files.replace(file_type_init, "")
    channel1 = Image.fromarray(np.uint16(channel1))
    channel1.save(str(folder_location + '/' + filename + '_' + stain1 + file_type_new))
    channel2 = Image.fromarray(np.uint16(channel2))
    channel2.save(str(folder_location + '/' + filename + '_' + stain2 + file_type_new))

*Step 6: Split the Images*

In [138]:
arr = os.listdir(folder_location)
file_list = np.asarray(arr)
file_list = folder_cleaner(file_list, file_type_new)

In [139]:
for files in file_list:
    image_slicer.slice(str(folder_location + '/' + files), slice_number)

*Moving the DAPI and Iba images into their own folders*

In [140]:
arr = os.listdir(folder_location)
file_list1 = np.asarray(arr)
file_list1 = folder_cleaner(file_list1, file_type_new)

In [141]:
for tiled_images in file_list1:
    conditional = str(str(tiled_images)[-5].isdigit())
    if conditional == 'True':
        if stain1 in tiled_images:
            shutil.move(str(folder_location + '/' + tiled_images), str(folder_location + '/' + stain1 + '/' + tiled_images))
        elif stain2 in tiled_images:
            shutil.move(str(folder_location + '/' + tiled_images), str(folder_location + '/' + stain2 + '/' + tiled_images))
    else:
        pass

*Step 4: Choose training and testing data sets*

In [142]:
arr = os.listdir(str(folder_location + '/' + stain1))
file_list_train = np.asarray(arr)
file_list_train = folder_cleaner(file_list_train, file_type_new)

In [143]:
X_train, X_test= train_test_split(file_list_train, test_size=0.20, random_state=random_state_num)

*Step X: Moving the testing and training DAPI data sets into test and train folders*

In [144]:
for names in file_list_train:
    if names in X_train[:]:
        shutil.move(str(folder_location + '/'+ stain1 + '/' + names), '/Users/hhelmbre/Desktop/train')
    else:
        shutil.move(str(folder_location + '/' + stain1 + '/' + names), '/Users/hhelmbre/Desktop/test')

*Step Y: Renaming the DAPI and Iba datasets according to proper VAMPIRE naming modality*

In [145]:
arr_train1 = os.listdir('/Users/hhelmbre/Desktop/train')
file_list_train1 = np.asarray(arr_train1)
file_list_train1 = folder_cleaner(file_list_train1, file_type_new)

In [146]:
arr_stain2 = os.listdir(str(folder_location + '/' + stain2))
file_list_stain2 = np.asarray(arr_stain2)
file_list_stain2 = folder_cleaner(file_list_stain2, file_type_new)

In [147]:
im_number= 1
for names in file_list_train1:
    dapi_name = str(names)

    
    
    if im_number < 10:
        os.rename(str('/Users/hhelmbre/Desktop/train/' + names), str('/Users/hhelmbre/Desktop/train/' + 'xy' + '0' + str(im_number) + 'c2.png'))
        
    else:
        os.rename(str('/Users/hhelmbre/Desktop/train/' + names), str('/Users/hhelmbre/Desktop/train/' + 'xy' + str(im_number) + 'c2.png'))
    
    iba_name = dapi_name.replace(stain1, stain2)
    
    if im_number < 10:
        os.rename(str(folder_location + '/' + stain2 + '/' + iba_name), str('/Users/hhelmbre/Desktop/train/' + 'xy' + '0' + str(im_number) + 'c1.png'))
        
    else:
        os.rename(str(folder_location + '/' + stain2 + '/' + iba_name), str('/Users/hhelmbre/Desktop/train/' + 'xy' + str(im_number) + 'c1.png'))
        
    
    im_number +=1

*Splitting the test group into the appropriate conditions*

In [148]:
arr_test = os.listdir('/Users/hhelmbre/Desktop/test')
file_list_test = np.asarray(arr_test)
file_list_test = folder_cleaner(file_list_test, file_type_new)

In [149]:
for test_images in file_list_test:
    if condition1 in test_images:
        shutil.move(str('/Users/hhelmbre/Desktop/test/' + test_images), str('/Users/hhelmbre/Desktop/test/' + condition1 + '/' + test_images))
    elif condition2 in test_images:
        shutil.move(str('/Users/hhelmbre/Desktop/test/' + test_images), str('/Users/hhelmbre/Desktop/test/' + condition2 + '/' + test_images))
    elif condition3 in test_images:
        shutil.move(str('/Users/hhelmbre/Desktop/test/' + test_images), str('/Users/hhelmbre/Desktop/test/' + condition3 + '/' + test_images))
    elif condition4 in test_images:
        shutil.move(str('/Users/hhelmbre/Desktop/test/' + test_images), str('/Users/hhelmbre/Desktop/test/' + condition4 + '/' + test_images))
    else:
        pass

*Step x: Renaming the test images and getting their appropriate iba stain*



In [150]:
arr_test_condition1 = os.listdir(str('/Users/hhelmbre/Desktop/test/' + condition1))
file_list_test_condition1 = np.asarray(arr_test_condition1)
file_list_test_condition1 = folder_cleaner(file_list_test_condition1, file_type_new)

In [151]:
arr_test_condition2 = os.listdir(str('/Users/hhelmbre/Desktop/test/' + condition2))
file_list_test_condition2 = np.asarray(arr_test_condition2)
file_list_test_condition2 = folder_cleaner(file_list_test_condition2, file_type_new)

In [152]:
arr_test_condition3 = os.listdir(str('/Users/hhelmbre/Desktop/test/' + condition3))
file_list_test_condition3 = np.asarray(arr_test_condition3)
file_list_test_condition3 = folder_cleaner(file_list_test_condition3, file_type_new)

In [153]:
arr_test_condition4 = os.listdir(str('/Users/hhelmbre/Desktop/test/' + condition4))
file_list_test_condition4 = np.asarray(arr_test_condition4)
file_list_test_condition4 = folder_cleaner(file_list_test_condition4, file_type_new)

In [154]:
im_number = 1
for names in file_list_test_condition1:
    dapi_name = str(names)

    
    if im_number < 10:
        os.rename(str('/Users/hhelmbre/Desktop/test/' + condition1 + '/'+ names), str('/Users/hhelmbre/Desktop/test/' + condition1 + '/' + 'xy' + '0' + str(im_number) + 'c2.png'))
        
    else:
        os.rename(str('/Users/hhelmbre/Desktop/test/' + condition1 + '/'+ names), str('/Users/hhelmbre/Desktop/test/' + condition1 + '/' + 'xy' + '0' + str(im_number) + 'c2.png'))
    
    iba_name = dapi_name.replace(stain1, stain2)
    
    if im_number < 10:
        os.rename(str(folder_location + '/' + stain2 + '/'+ iba_name), str('/Users/hhelmbre/Desktop/test/' + condition1 + '/' + 'xy' + '0' + str(im_number) + 'c1.png'))
        
    else:
        os.rename(str(folder_location + '/' + stain2 + '/'+ iba_name), str('/Users/hhelmbre/Desktop/test/' + condition1 + '/' + 'xy' + '0' + str(im_number) + 'c1.png'))
        
    
    im_number +=1

In [155]:
im_number= 1
for names in file_list_test_condition2:
    dapi_name = str(names)

    
    if im_number < 10:
        os.rename(str('/Users/hhelmbre/Desktop/test/' + condition2 + '/'+ names), str('/Users/hhelmbre/Desktop/test/' + condition2 + '/' + 'xy' + '0' + str(im_number) + 'c2.png'))
        
    else:
        os.rename(str('/Users/hhelmbre/Desktop/test/' + condition2 + '/'+ names), str('/Users/hhelmbre/Desktop/test/' + condition2 + '/' + 'xy' + '0' + str(im_number) + 'c2.png'))
    
    iba_name = dapi_name.replace(stain1, stain2)
    
    if im_number < 10:
        os.rename(str(folder_location + '/' + stain2 + '/'+ iba_name), str('/Users/hhelmbre/Desktop/test/' + condition2 + '/' + 'xy' + '0' + str(im_number) + 'c1.png'))
        
    else:
        os.rename(str(folder_location + '/' + stain2 + '/'+ iba_name), str('/Users/hhelmbre/Desktop/test/' + condition2 + '/' + 'xy' + '0' + str(im_number) + 'c1.png'))
        
    
    im_number +=1

In [156]:
im_number= 1
for names in file_list_test_condition3:
    dapi_name = str(names)

    
    if im_number < 10:
        os.rename(str('/Users/hhelmbre/Desktop/test/' + condition3 + '/'+ names), str('/Users/hhelmbre/Desktop/test/' + condition3 + '/' + 'xy' + '0' + str(im_number) + 'c2.png'))
        
    else:
        os.rename(str('/Users/hhelmbre/Desktop/test/' + condition3 + '/'+ names), str('/Users/hhelmbre/Desktop/test/' + condition3 + '/' + 'xy' + '0' + str(im_number) + 'c2.png'))
    
    iba_name = dapi_name.replace(stain1, stain2)
    
    if im_number < 10:
        os.rename(str(folder_location + '/' + stain2 + '/'+ iba_name), str('/Users/hhelmbre/Desktop/test/' + condition3 + '/' + 'xy' + '0' + str(im_number) + 'c1.png'))
        
    else:
        os.rename(str(folder_location + '/' + stain2 + '/'+ iba_name), str('/Users/hhelmbre/Desktop/test/' + condition3 + '/' + 'xy' + '0' + str(im_number) + 'c1.png'))
        
    
    im_number +=1

In [157]:
im_number= 1
for names in file_list_test_condition4:
    dapi_name = str(names)

    
    if im_number < 10:
        os.rename(str('/Users/hhelmbre/Desktop/test/' + condition4 + '/'+ names), str('/Users/hhelmbre/Desktop/test/' + condition4 + '/' + 'xy' + '0' + str(im_number) + 'c2.png'))
        
    else:
        os.rename(str('/Users/hhelmbre/Desktop/test/' + condition4 + '/'+ names), str('/Users/hhelmbre/Desktop/test/' + condition4 + '/' + 'xy' + '0' + str(im_number) + 'c2.png'))
    
    iba_name = dapi_name.replace(stain1, stain2)
    
    if im_number < 10:
        os.rename(str(folder_location + '/' + stain2 + '/'+ iba_name), str('/Users/hhelmbre/Desktop/test/' + condition4 + '/' + 'xy' + '0' + str(im_number) + 'c1.png'))
        
    else:
        os.rename(str(folder_location + '/' + stain2 + '/'+ iba_name), str('/Users/hhelmbre/Desktop/test/' + condition4 + '/' + 'xy' + '0' + str(im_number) + 'c1.png'))
        
    
    im_number +=1